In [2]:
!pip install -q -U keras-nlp tensorflow-text
!pip install -q -U tensorflow-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.18.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import jax
jax.devices()

E0000 00:00:1736461703.177160      10 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
E0109 22:28:23.209034546      10 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2025-01-09T22:28:23.209016143+00:00"}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

- Keras is a high level multi-framework deeplearning API designed for simplicity and ease of use,Using keras 3 you can run workflows on one of the three backends: Tensorflow ,Jax or pytorch

In [4]:
import os
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

Install dependencies<br>
Install Keras, KerasNLP, and other dependencies.

In [5]:
import keras 
import keras_nlp

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Keras Distribute: https://keras.io/guides/distribution/

In [6]:
## !! Creating a device mesh with (1, number of devices) so that weights are sharded across
## CHANGE HERE FOR TPU 1
device_mesh = keras.distribution.DeviceMesh(
    (1,8),
    ["batch","model"],
    devices = keras.distribution.list_devices(),
)

In [7]:
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)
layout_map["token_embedding/embeddings"] = (model_dim,None)

layout_map["decoder_block.*attenetion.*(query|key|value)/kernel"] = (model_dim,None,None)
layout_map["decoder_block.*attention_output/kernel"]= (model_dim,None,None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None,model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim,None)

In [8]:
def remove_surrogates(text):
    return ''.join(char for char in text if not (0xD800 <= ord(char) <= 0xDFFF))

In [9]:
import pandas as pd
input_columns = ['prompt' , 'response_a','response_b']
label_columns = ['winner_model_a','winner_model_b']

raw_train_dataset = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet')


In [10]:
raw_train_dataset.head(5)

,id,prompt,response_a,response_b,winner,model_a,model_b,language
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English


In [11]:
def to_onehot(row):
    # Return the row with new columns added
    return pd.Series({
        "winner_model_a": 1 if row['winner'] == "model_a" else 0,
        "winner_model_b": 1 if row['winner'] == "model_b" else 0
    })

# Apply and add new columns
onehot_encoded = raw_train_dataset.apply(to_onehot, axis=1)

# Add the one-hot encoded columns to the original DataFrame
raw_train_dataset = pd.concat([raw_train_dataset, onehot_encoded], axis=1)


In [12]:
raw_train_dataset.head(5)

,id,prompt,response_a,response_b,winner,model_a,model_b,language,winner_model_a,winner_model_b
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak,1,0
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian,1,0
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish,1,0
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English,0,1
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English,1,0


In [13]:
# raw_train_dataset[input_columns] = raw_train_dataset[input_columns].map(lambda x: eval(x)[0] if 'null' not in x else None)

In [14]:
raw_train_dataset = raw_train_dataset.dropna().drop(['model_a','model_b'],axis=1).reset_index(drop=True)

In [15]:
train_dataset = pd.DataFrame({
    'text': raw_train_dataset[input_columns].apply(
        lambda row: (
            f"\n\nPROMPT:\n\n{row['prompt']}\n\n"
            f"RESPONSE A:\n\n{row['response_a']}\n\n"
            f"RESPONSE B:\n\n{row['response_b']}\n\n"
            "TASK: select the model that gave better response model_a or model_b just output in one word"
        ), axis=1).apply(remove_surrogates),
    'label': raw_train_dataset[label_columns].apply(lambda x: x.values.tolist(), axis=1)
})


In [16]:
train_dataset.head()

,text,label
0,\n\nPROMPT:\n\nvieš po Slovensky?\n\nRESPONSE ...,"[1, 0]"
1,\n\nPROMPT:\n\nYou will be given a piece of ne...,"[1, 0]"
2,"\n\nPROMPT:\n\nDört basamaklı, rakamları birbi...","[1, 0]"
3,\n\nPROMPT:\n\n현재 추천된 탑 3 종목인 Cabaletta Bio (C...,"[0, 1]"
4,\n\nPROMPT:\n\nPlease be boring \n\nRESPONSE A...,"[1, 0]"


In [17]:
model_parallel = keras.distribution.ModelParallel(
    layout_map = layout_map,
    batch_dim_name = "batch",
)

keras.distribution.set_distribution(model_parallel)

#### Load Model
KerasNLP provides implementations of many popular model architectures. In this tutorial, you'll create a model using GemmaCausalLM, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.
<br>
Create the model using the from_preset method:

In [18]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma2/keras/gemma2_instruct_9b_en/3")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,241,705,984 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,241,705,984 (34.43 GB)

 Trainable params: 9,241,705,984 (34.43 GB)

 Non-trainable params: 0 (0.00 B)

#### LoRA Fine-tuning
- To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA)

- The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

- A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

In [19]:
gemma_lm.backbone.enable_lora(rank = 16)

In [20]:
for layer in gemma_lm._backbone.layers[:16]:
    layer.trainable = False

In [21]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,299,852,800 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,299,852,800 (34.64 GB)

 Trainable params: 40,148,992 (153.16 MB)

 Non-trainable params: 9,259,703,808 (34.50 GB)

In [22]:
def preprocess_fn(text, label=None):
    preprocessed = gemma_lm._preprocessor(text, sequence_length=1024)[0]
    # Ensure the preprocess function returns only the necessary inputs
    return {'token_ids' : preprocessed['token_ids'], 'padding_mask' : preprocessed['padding_mask']}, label if label is not None else text

In [23]:
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling1D
from keras import Model

inputs = {
    "token_ids": keras.Input(shape=(1024,), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(1024,), dtype=tf.int32, name="padding_mask"),
}
x = gemma_lm.backbone(inputs)
print(x.shape)
x = GlobalAveragePooling1D()(x)
print(x.shape)

outputs = Dense(2, 'softmax')(x)
model = Model(inputs, outputs)

(None, 1024, 3584)
(None, 3584)


In [24]:
optimizer = keras.optimizers.AdamW(
                learning_rate=5e-5,
                weight_decay=0.01,)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

In [25]:
model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(),)

In [26]:
import tensorflow as tf
ds = tf.data.Dataset.from_tensor_slices((train_dataset.text.values, raw_train_dataset[label_columns].values)).batch(2).map(preprocess_fn)
ds = ds.shuffle(ds.cardinality())

In [27]:
train_split = ds.take(int(len(ds)*0.9))
val_split = ds.skip(int(len(ds)*0.9)).take(int(len(ds)*0.1))
histories = model.fit(train_split, validation_data=[val_split], epochs=2, batch_size=2)

Epoch 1/2
21798/21798 ━━━━━━━━━━━━━━━━━━━━ 10666s 478ms/step - loss: 0.7337 - val_loss: 0.6449
Epoch 2/2
21798/21798 ━━━━━━━━━━━━━━━━━━━━ 10512s 474ms/step - loss: 0.6164 - val_loss: 0.5075


In [28]:
import numpy as np
layer = model.get_layer(name='dense')
weights = layer.get_weights()
kernel, bias = weights

# Save the kernel and bias separate
np.save('dense_1_kernel.npy', kernel)
np.save('dense_1_bias.npy', bias)
model.layers[2].save_lora_weights("model.lora.h5")

In [32]:
#learning about the llm fine tuning

In [33]:
# It is observed that:
'''
- the model is taking 3 hrs time for each epoch
- keep persistence on 
- keep suitable number of epochs so that it dont exceed session timeout 

'''

'\n- the model is taking 3 hrs time for each epoch\n- keep persistence on \n- keep suitable number of epochs so that it dont exceed session timeout \n\n'

In [ ]:
import shutil

# Compress the /kaggle/working directory into a zip file
shutil.make_archive('/kaggle/working/all_files', 'zip', '/kaggle/working')

# Output the zip file for download
from IPython.display import FileLink
FileLink('/kaggle/working/all_files.zip')
